In [ ]:
import os
import re
import pandas as pd
import cx_Oracle

In [ ]:
# Config
dsn_tns = cx_Oracle.makedsn('xxxxx', '1521', service_name='xxxxx') # if needed, place an 'r' before any parameter in order to address special characters such as '\'.
conn = cx_Oracle.connect(user=r'xxxx', password='xxxx', dsn=dsn_tns) # if needed, place an 'r' before any parameter in order to address special characters such as '\'. For example, if your user name contains '\', you'll need to place 'r' before the user name: user=r'User Name'

In [ ]:
# Provide tablenames as list (uppercase table names for Oracle)
# if you need all tables in a schema, run a query and get it to a pandas dataframe

table_names = ['TABLE1', 'TABLE2']

for table_name in table_names:
    sql = (f"SELECT TABLE_NAME, COLUMN_NAME, COLUMN_ID, DATA_TYPE, DATA_PRECISION, DATA_SCALE, \
    CASE WHEN DATA_TYPE = 'NUMBER' AND DATA_SCALE = 0 THEN 'YES' \
         WHEN DATA_TYPE = 'NUMBER' AND COLUMN_NAME LIKE '%_KEY' THEN 'YES' ELSE 'NO' END AS IS_INTEGER \
    FROM ALL_TAB_COLUMNS WHERE TABLE_NAME = '{table_name}' ORDER BY 1, 3")
    
    df = pd.read_sql(sql, con=conn)   
    print(table_name)
    sample = []
    cols = []
    for index, row in df.iterrows():
        if row['DATA_TYPE'] == 'DATE':
            sample.append('1900/01/01 00:00:00')
        if ((row['DATA_TYPE'] == 'VARCHAR2') or row['DATA_TYPE'] == 'CHAR'):
            sample.append('-')
        if (row['DATA_TYPE'] == 'NUMBER' and row['IS_INTEGER'] == 'YES'):
        sample.append(0)
        if (row['DATA_TYPE'] == 'NUMBER' and row['IS_INTEGER'] == 'NO'):
            sample.append(0.1)      
        cols.append(row['COLUMN_NAME'])    
    df['Sample'] = sample        

    res = df.pivot(index = 'TABLE_NAME', columns = 'COLUMN_ID', values='Sample')
    res.reset_index()
    res.to_csv(table_name+'.csv', index=False, header = cols)

In [ ]:
# Adhoc Schema File Generation
table_name = 'ADHOC_TBL1'

sql = (f"SELECT TABLE_NAME, COLUMN_NAME, COLUMN_ID, DATA_TYPE, DATA_PRECISION, DATA_SCALE, \
CASE WHEN DATA_TYPE = 'NUMBER' AND DATA_SCALE = 0 THEN 'YES' \
     WHEN DATA_TYPE = 'NUMBER' AND COLUMN_NAME LIKE '%_KEY' THEN 'YES' ELSE 'NO' END AS IS_INTEGER \
FROM ALL_TAB_COLUMNS WHERE TABLE_NAME = '{table_name}' ORDER BY 1, 3")

df = pd.read_sql(sql, con=conn)

sample = []
cols = []
for index, row in df.iterrows():
    if row['DATA_TYPE'] == 'DATE':
        sample.append('1900/01/01 00:00:00')
    if ((row['DATA_TYPE'] == 'VARCHAR2') or row['DATA_TYPE'] == 'CHAR'):
        sample.append('-')
    if (row['DATA_TYPE'] == 'NUMBER' and row['IS_INTEGER'] == 'YES'):
        sample.append(0)
    if (row['DATA_TYPE'] == 'NUMBER' and row['IS_INTEGER'] == 'NO'):
        sample.append(0.1)
    cols.append(row['COLUMN_NAME'])
df['Sample'] = sample

res = df.pivot(index = 'TABLE_NAME', columns = 'COLUMN_ID', values='Sample')
res.reset_index()
res.to_csv(table_name+'.csv', index=False, header = cols)